This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.2_tutorials/wallaroo-run-anywhere/model-management/inline-edge-model-replacements-tutorial).

## In-Line Model Updates at the Edge Tutorial

Wallaroo pipeline publishes are containerized versions of the pipeline, models and model steps, and the inference engine published to an Open Container Initiative (OCI) Registry.  Once published, they are used to deploy models to edge devices and used for inference requests.

Edge locations added to a pipeline published allow deployments on edge devices to connect with the Wallaroo Ops instance and transmit their inference results as part of the pipeline logs.

Pipeline publishes can be replaced by new pipeline publishes from the following sources:

* A new version of the same pipeline that creates a new pipeline publish.  In these instances, the edge locations associated with the pipeline publish are preserved with the original pipeline along with their pipeline inference logs.  Any input or output schema changes from the new models or models steps are reflected in the pipeline logs.
* A separate pipeline with its own models and model steps.  In this scenario, the edge locations associated with the original pipeline publish are assigned to the new pipeline.  The inference logs are stored with the original pipeline, and new inference logs for the edge locations are stored as part of the new pipeline.

For more information on pipeline inference logs, see [Inference Logs](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-serve/wallaroo-model-operations-service-pipeline-logs/).

* **IMPORTANT NOTE**:  Assays and other log data is based on the pipeline and model name.  If these values change, assays that are currently running will no longer generate analyses.  These should be paused, and new assays based on the current pipeline and model name should be created.  For more information, see [Model Drift Detection with Model Insights](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-observe/wallaroo-pipeline-assays/). 

## Goal

* Package a model in Wallaroo and perform sample inferences.  Publish the pipeline and create an edge location, then deploy the model to an edge location.
* Perform sample inferences on the edge location and verify the inference logs are added to the pipeline associated with the edge location.
* Create a new pipeline version with a new model step and perform sample inferences.  Update the publish with the new pipeline version, and verify that the model is deployed to the edge location deployment.
* Perform sample inferences on the edge location to verify the new model is being used, and verify the inference logs are added to the pipeline associated with the edge location.
* Create a new pipeline with a different model and perform sample inferences.  Publish the pipeline, then replace the publish used for the edge location.  Verify that the edge location is transferred to the new pipeline.
* Perform sample inferences on the edge location to verify model is updated from the new pipeline is being used, and verify the inference logs are added to the new pipeline associated with the edge location.

### Resources

This tutorial provides the following:

* Models:
  * `models/rf_model.onnx`: The champion model that predicts house prices.
  * `models/xgb_model.onnx`: A new challenger model that takes the same inputs as `models/rf_model.onnx` and outputs house prices with a field of the same name.
  * `models/gbr_model.onnx`: A new challenger model that takes the same inputs as `models/rf_model.onnx` and outputs house prices with a field of the same name.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.


## Edge Deployment Preparation Steps

The following is used to set up the environment that is used for the pipeline publish replacement and in-line model updates on an edge tutorial.

In these steps, we will:

* Deploying a sample ML model used to determine house prices based on a set of input parameters.
* Publish the model deployment configuration to an OCI registry.
* Use the publish and set edge locations.
* Deploy the model in an edge location and perform sample inferences.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

import json
import time

workspace_name = 'in-line-edge-replacement-demo'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace names are unique across the Wallaroo instance.  If a user requests a workspace they have not been granted access to, an error message is returned.

In [3]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

{'name': 'in-line-edge-replacement-demo', 'id': 1450, 'archived': False, 'created_by': 'preethi.kumar@wallaroo.ai', 'created_at': '2025-01-03T20:50:31.056555+00:00', 'models': [{'name': 'rf-house-price-estimator', 'versions': 4, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 1, 7, 6, 5, 53, 129118, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 1, 3, 20, 50, 34, 629031, tzinfo=tzutc())}], 'pipelines': [{'name': 'edge-inline-replacement-demon', 'create_time': datetime.datetime(2025, 1, 3, 20, 50, 47, 990926, tzinfo=tzutc()), 'definition': '[]'}]}

### Upload The Champion Model

For our example, we will upload the champion model that has been trained to derive house prices from a variety of inputs.  The model file is `rf_model.onnx`, and is uploaded with the name `rf-house-price-estimator`.

In [4]:
housing_model_control = (wl.upload_model("rf-house-price-estimator", 
                                        './models/rf_model.onnx', 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

### Build and Deploy the Pipeline

This pipeline is made to be an example of an existing situation where a model is deployed and being used for inferences in a production environment.  We'll call it `edge-inline-replacement-demonstration`, set `housing_model_control` as a pipeline step, then run a sample inference.

In [5]:
mainpipeline = wl.build_pipeline('edge-inline-replacement-demon')

# undeploy if already created earlier
mainpipeline.undeploy()
# clear the steps if used before
mainpipeline.clear()

mainpipeline.add_model_step(housing_model_control)

 ok


name,edge-inline-replacement-demon
created,2025-01-03 20:50:47.990926+00:00
last_updated,2025-07-11 19:21:41.186582+00:00
deployed,False
workspace_id,1450
workspace_name,in-line-edge-replacement-demo
arch,x86
accel,none
tags,
versions,"ef7d1c53-beec-49d5-954d-2fff4294161e, f3fa996d-ae1e-401e-8f8c-98c89e2c7009, eb5114c2-5107-47f4-8531-b7dbf7f9118e, 28314a82-eb40-4779-9801-8a8ee3e7227f, a1392690-c205-4524-8e0f-e0e1e673af72, 1aacdc72-671d-4be6-8aa4-91fea99a6c97, fdef8fb8-a4a5-491a-9f2c-0c32bffee0aa, f897a4b6-d768-47ef-9fad-9fc36bf0fb93, c84e3f55-f963-4118-a3c7-f00d31ab2b7f, cf93e8a8-b586-40e4-b5fa-2997feb39913, 086f04b1-dbe9-46de-bef1-54115bd9fc25, 7f36f537-2155-4240-843a-784eee0697e8, 9ee8b6f3-9e19-45e4-8cbf-8dcbdf4813c7"
steps,rf-house-price-estimator


In [6]:
#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

mainpipeline.deploy(deployment_config = deploy_config)

Waiting for deployment - this will take up to 45s ............ ok


name,edge-inline-replacement-demon
created,2025-01-03 20:50:47.990926+00:00
last_updated,2025-07-11 19:21:42.251965+00:00
deployed,True
workspace_id,1450
workspace_name,in-line-edge-replacement-demo
arch,x86
accel,none
tags,
versions,"2da8ce7c-4da0-404a-aafe-f0af99d5ce15, ef7d1c53-beec-49d5-954d-2fff4294161e, f3fa996d-ae1e-401e-8f8c-98c89e2c7009, eb5114c2-5107-47f4-8531-b7dbf7f9118e, 28314a82-eb40-4779-9801-8a8ee3e7227f, a1392690-c205-4524-8e0f-e0e1e673af72, 1aacdc72-671d-4be6-8aa4-91fea99a6c97, fdef8fb8-a4a5-491a-9f2c-0c32bffee0aa, f897a4b6-d768-47ef-9fad-9fc36bf0fb93, c84e3f55-f963-4118-a3c7-f00d31ab2b7f, cf93e8a8-b586-40e4-b5fa-2997feb39913, 086f04b1-dbe9-46de-bef1-54115bd9fc25, 7f36f537-2155-4240-843a-784eee0697e8, 9ee8b6f3-9e19-45e4-8cbf-8dcbdf4813c7"
steps,rf-house-price-estimator


### Inference Request Testing

We'll use a single row inference request that should return a result of around `$700k`.

In [7]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 
                                                      2.5, 
                                                      2900.0, 
                                                      5505.0, 
                                                      2.0, 
                                                      0.0, 
                                                      0.0, 
                                                      3.0, 
                                                      8.0, 
                                                      2900.0, 
                                                      0.0, 
                                                      47.6063, 
                                                      -122.02, 
                                                      2970.0, 
                                                      5251.0, 
                                                      12.0, 
                                                      0.0, 
                                                      0.0]]})
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2025-07-11 19:21:56.632,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0


### Edge Deployment

We can now deploy the original pipeline to an edge device.  This will require the following steps:

* Publish the pipeline:  Publishes the pipeline to the OCI registry.
* Add Edge:  Add the edge location to the pipeline publish.
* Deploy Edge:  Deploy the edge device with the edge location settings.

#### Publish Pipeline

Publishing the pipeline uses the pipeline `wallaroo.pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [8]:
pub = mainpipeline.publish()

Waiting for pipeline publish... It may take up to 600 sec.
... Published.blishing.


#### Add Edge Location

The edge location is added with the [`wallaroo.pipeline_publish.add_edge(name)`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/pipeline_publish/#PipelinePublish.add_edge) method.  This returns the OCI registration information, and the `EDGE_BUNDLE` information.  The `EDGE_BUNDLE` data is a base64 encoded set of parameters for the pipeline that the edge device is associated with, the workspace, and other data.

For full details, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication: Edge Observability](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/#edge-observability).

For this example, we will add two locations:

* `houseprice-edge-inline-demo`

These will be used in later steps for demonstrating inferences through different locations.

In [10]:
edge_name_01 = "houseprice-edge-inline-demo"
edge_publish_01 = pub.add_edge(edge_name_01)
display(edge_publish_01)

ID,85
Pipeline Name,edge-inline-replacement-demon
Pipeline Version,f028d883-6216-4c0e-baeb-7dc646fa2437
Status,Published
Workspace Id,1450
Workspace Name,in-line-edge-replacement-demo
Edges,houseprice-edge-inline-demo
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-inline-replacement-demon:f028d883-6216-4c0e-baeb-7dc646fa2437
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-inline-replacement-demon
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:c383d2332a9da6d59618087b04b5837c2a7878caa444848f25157e6092dfe153


#### DevOps Deployment

The following shows two methods of model deployments on edge devices.  For more details on model edge deployments using Docker or `helm` with Wallaroo, see [Model Operations: Run Anywhere](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

##### Docker Run Based Deployment

For the model deployment to the edge location via `docker run`, use the `Docker Run Command` displayed with the pipeline publish.    The following variables must be set first.

* `PERSISTENT_VOLUME_DIR`: The file path for the persistent volume to store session data.
* `EDGE_PORT`: The external port to connect to, typically `8080`.
* `OCI_USERNAME`: The username for the OCI registry service.
* `OCI_PASSWORD`: The password or token for the OCI registry service.
* `

Before deploying the pipeline, verify that Docker is able to connect to the OCI registry with the following command.

```bash
docker login -u $OCI_USERNAME --password $OCI_PASSWORD (Your registry URL)
```

#### Edge Inference Example

We will perform sample inferences on the **current** model edge deployment to demonstrate how the inferences change as the models are updated through the in-line edge deployments.

With the model deployed on the edge location, we will perform sample inferences on the edge deployment.  For this example, the hostname is `testboy.local`.  Adjust the hostname to fit your edge location deployment.

The following endpoints are available.

| Endpoint | Type | Description |
|---|---|---|
| `/models` | GET | Returns the models and model versions used in the edge deployment. |
| `/pipelines` | GET | Returns the pipeline ID and status of the pipeline used in the edge deployment. |
| `/infer` | POST | The inference request endpoint.  This accepts either an Apache Arrow table, or a JSON in pandas record format.

For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

First we'll retrieve the model.  We should see the same name we set when we uploaded it to the Wallaroo instance.

In [10]:
!curl testboy.local:8080/models

{"models":[{"sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6","name":"rf-house-price-estimator","version":"02f05eac-5b37-4784-84a4-5db7ab95b65b","status":"Running"}]}

Now we'll retrieve the pipeline details, and verify the name is the same as set earlier and it is running.

In [11]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-inline-replacement-demon","version":"7c702eca-8acf-45e0-bdcd-bbb48a5102e5","status":"Running"}]}

Now do the infer with the same DataFrame we used for our sample inference when the model was deployed in the Wallaroo instance.

We use the endpoint `/infer`; this endpoint remains the same even if the models and the pipeline changes, which is demonstrated in the later steps.

In [12]:
import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 
                                            2.5, 
                                            2900.0, 
                                            5505.0, 
                                            2.0, 
                                            0.0, 
                                            0.0, 
                                            3.0, 
                                            8.0, 
                                            2900.0, 
                                            0.0, 
                                            47.6063, 
                                            -122.02, 
                                            2970.0, 
                                            5251.0, 
                                            12.0, 
                                            0.0, 
                                            0.0]]})

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}' > ./inferenceoutput.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput.df.json', orient="records")
df_result.loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   628  100   500  100   128   1811    463 --:--:-- --:--:-- --:--:--  2283


,time,out,metadata
0,1713800294502,{'variable': [718013.7]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': '7c702eca-8acf-45e0-bdcd-bbb48a5102e5', 'elapsed': [142461514, 55185536], 'dropped': [], 'partition': 'houseprice-edge-inline-demonstration'}"


## How to Replace a Pipeline Publish

Pipeline publishes are replaced with the `wallaroo.pipeline.Pipeline.publish()` method.  This creates a new publish from the most recent pipeline version and takes the following parameters.

| Parameter | Type | Description |
|---|---|---|
| **deployment_config** | *wallaroo.deployment_config.DeploymentConfig* (*Optional*) | The deployment configuration used for the edge deployment.  By default, this is the same deployment configuration used for the pipeline. |
| **replaces** | *[List[wallaroo.pipeline_publish]]* (*Optional*) | The pipeline publish(es) to replace.|

When a pipeline published is replaced with a new one, the edge locations are transferred to the pipeline that the publish came from.  In this example, this is the same pipeline.  Inference results from the edge location deployments are stored with the pipeline that generated the publish.  Note that if the model or pipeline steps have changed from one pipeline version to the next, the pipeline log schema will change with it.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Log Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-logs/).

### Verify Pipeline Publishes and Edge Locations

We verify that the pipeline is published and is assigned edge locations via the `wallaroo.pipeline.Pipeline.publishes()` method and `wallaroo.pipeline.Pipeline.list_edges()` method.

In [11]:
display('Publishes:')
display(mainpipeline.publishes())
display('Edge Locations')
display(mainpipeline.list_edges())

'Publishes:'

id,Pipeline Name,Pipeline Version,Workspace Id,Workspace Name,Edges,Engine URL,Pipeline URL,Created By,Created At,Updated At
19,edge-inline-replacement-demon,086f04b1-dbe9-46de-bef1-54115bd9fc25,1450,in-line-edge-replacement-demo,houseprice-edge-inline-demonstration,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.4.0-5866,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-inline-replacement-demon:086f04b1-dbe9-46de-bef1-54115bd9fc25,preethi.kumar@wallaroo.ai,2025-03-Jan 20:51:14,2025-03-Jan 20:51:14
20,edge-inline-replacement-demon,f897a4b6-d768-47ef-9fad-9fc36bf0fb93,1450,in-line-edge-replacement-demo,my-edge-demo,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.4.0-5873,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-inline-replacement-demon:f897a4b6-d768-47ef-9fad-9fc36bf0fb93,preethi.kumar@wallaroo.ai,2025-06-Jan 18:27:42,2025-06-Jan 18:27:42
21,edge-inline-replacement-demon,a1392690-c205-4524-8e0f-e0e1e673af72,1450,in-line-edge-replacement-demo,my-edge-demo-1,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.4.0-5873,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-inline-replacement-demon:a1392690-c205-4524-8e0f-e0e1e673af72,preethi.kumar@wallaroo.ai,2025-07-Jan 01:20:40,2025-07-Jan 01:20:40
22,edge-inline-replacement-demon,f3fa996d-ae1e-401e-8f8c-98c89e2c7009,1450,in-line-edge-replacement-demo,my-edge-demo-2,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.4.0-5875,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-inline-replacement-demon:f3fa996d-ae1e-401e-8f8c-98c89e2c7009,preethi.kumar@wallaroo.ai,2025-07-Jan 06:05:58,2025-07-Jan 06:05:58
85,edge-inline-replacement-demon,f028d883-6216-4c0e-baeb-7dc646fa2437,1450,in-line-edge-replacement-demo,houseprice-edge-inline-demo,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-inline-replacement-demon:f028d883-6216-4c0e-baeb-7dc646fa2437,john.hummel@wallaroo.ai,2025-11-Jul 19:21:56,2025-11-Jul 19:21:56


'Edge Locations'

ID,Name,Publish ID,Created At,Tags,CPUs,Memory,SPIFFE ID
ca6018e1-2239-4511-958f-2aadb88cc92f,houseprice-edge-inline-demonstration,19,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/ca6018e1-2239-4511-958f-2aadb88cc92f
51fe06d6-d194-410c-ade5-8d9a60bd3da6,my-edge-demo,20,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/51fe06d6-d194-410c-ade5-8d9a60bd3da6
20db9fdb-2dde-4a18-bb75-2d62343d0a28,my-edge-demo-1,21,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/20db9fdb-2dde-4a18-bb75-2d62343d0a28
57a1dc90-4baf-4548-ae33-e3186b5672ea,my-edge-demo-2,22,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/57a1dc90-4baf-4548-ae33-e3186b5672ea
619aefe2-5c23-48e1-a256-fc807d452d0f,houseprice-edge-inline-demo,85,2025-07-11T19:22:21.621626+00:00,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/619aefe2-5c23-48e1-a256-fc807d452d0f


### How to Replace a Pipeline Publish from a New Pipeline Version

The following demonstrates replacing a pipeline publish with a new publish from a pipeline version.  In this procedure, the edge location has an existing model edge deployment.  The publish that is used for that deployment is replaced, and an in-line update of the model deployment is performed on the edge device.

#### Create New Pipeline Version

For the next demonstration, we will use the same pipeline and update the pipeline steps with a new model.  This time with the model `./models/xgb_model.onnx`, which takes the same input as `./models/rf_model.onnx`.  We will also set a different name for this model to distinguish it from the one previously uploaded and used as the pipeline step.

Once set, we will deploy the pipeline with the new model, and perform a new inference request.  From this, we will see that this new model outputs a slightly different prediction that the previous one - closer to `$650k`.

In [12]:
housing_model_challenger01 = (wl.upload_model("xgb-house-price-estimator", 
                                        './models/xgb_model.onnx', 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

# undeploy if already created earlier
mainpipeline.undeploy()
# clear the steps if used before
mainpipeline.clear()

mainpipeline.add_model_step(housing_model_challenger01)

Waiting for undeployment - this will take up to 45s ....................................... ok


name,edge-inline-replacement-demon
created,2025-01-03 20:50:47.990926+00:00
last_updated,2025-07-11 19:21:56.673646+00:00
deployed,False
workspace_id,1450
workspace_name,in-line-edge-replacement-demo
arch,x86
accel,none
tags,
versions,"f028d883-6216-4c0e-baeb-7dc646fa2437, 2da8ce7c-4da0-404a-aafe-f0af99d5ce15, ef7d1c53-beec-49d5-954d-2fff4294161e, f3fa996d-ae1e-401e-8f8c-98c89e2c7009, eb5114c2-5107-47f4-8531-b7dbf7f9118e, 28314a82-eb40-4779-9801-8a8ee3e7227f, a1392690-c205-4524-8e0f-e0e1e673af72, 1aacdc72-671d-4be6-8aa4-91fea99a6c97, fdef8fb8-a4a5-491a-9f2c-0c32bffee0aa, f897a4b6-d768-47ef-9fad-9fc36bf0fb93, c84e3f55-f963-4118-a3c7-f00d31ab2b7f, cf93e8a8-b586-40e4-b5fa-2997feb39913, 086f04b1-dbe9-46de-bef1-54115bd9fc25, 7f36f537-2155-4240-843a-784eee0697e8, 9ee8b6f3-9e19-45e4-8cbf-8dcbdf4813c7"
steps,rf-house-price-estimator


In [13]:
#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

mainpipeline.deploy(deployment_config = deploy_config)
mainpipeline.steps()

Waiting for deployment - this will take up to 45s ............ ok


[{'ModelInference': {'models': [{'name': 'xgb-house-price-estimator', 'version': 'f610c416-2536-46c8-b54c-723349866555', 'sha': '31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c'}]}}]

In [14]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 
                                                      2.5, 
                                                      2900.0, 
                                                      5505.0, 
                                                      2.0, 
                                                      0.0, 
                                                      0.0, 
                                                      3.0, 
                                                      8.0, 
                                                      2900.0, 
                                                      0.0, 
                                                      47.6063, 
                                                      -122.02, 
                                                      2970.0, 
                                                      5251.0, 
                                                      12.0, 
                                                      0.0, 
                                                      0.0]]})
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2025-07-11 19:23:33.645,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0


#### Replace Pipeline Publish with new Pipeline Version

From our pipeline, we will publish the new version.  The method `wallaroo.pipeline.Pipeline.publish()` creates a new publish from the most recent pipeline version and takes the following parameters.

We will replace our publish earlier, labeled `pub`, with the new publish generated from the pipeline labeled `new_pub`.

The details on deploying the replaced pipeline publish(es) is under the `Replaces` section.

In [15]:
new_pub = mainpipeline.publish(replaces=[pub])
new_pub

Waiting for pipeline publish... It may take up to 600 sec.
... Published.blishing.


ID,86
Pipeline Name,edge-inline-replacement-demon
Pipeline Version,f75a1a83-a783-48b8-9b0f-151dbd7564a7
Status,Published
Workspace Id,1450
Workspace Name,in-line-edge-replacement-demo
Edges,houseprice-edge-inline-demo
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-inline-replacement-demon:f75a1a83-a783-48b8-9b0f-151dbd7564a7
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-inline-replacement-demon
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:811576242dbb252f462e591d2150a297f725bb6b88681bdb2c2c516676cc10f5


Note that in the `Replaces` section, updates are given for each edge location.  On the edge device, we deploy the new pipeline publish either by the provided **Docker Run Command** or **Helm Install Command**.  Note that the **Helm Install Command** uses `helm upgrade` rather than `helm install` to create a new revision.  The pipeline steps and models will be changed at the model edge location.

Before we do so, we will verify the edge locations associated for our pipelines.  `mainpipeline` will still have its edges, since we only updated the **pipeline version**, while `new_pipeline` has no edges yet.

#### Inference Requests from Replaced Pipeline Version Deployment Examples

We deploy the in-line model update in our new publish above for the edge location, then perform the same checks of the models, pipeline, then a sample inference using the same inference input data as before.

We will see the new model name and model version used.

In [18]:
!curl testboy.local:8080/models

{"models":[{"sha":"31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c","name":"xgb-house-price-estimator","version":"a86668be-03ed-47f8-8f49-9e31fe502566","status":"Running"}]}

In [19]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-inline-replacement-demon","version":"c6a1e945-7de0-4f2c-addb-4f4746114a86","status":"Running"}]}

We now perform the same inference request on the edge location deployment, and show the results match that as the pipeline with the new model.

In [20]:
import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 
                                            2.5, 
                                            2900.0, 
                                            5505.0, 
                                            2.0, 
                                            0.0, 
                                            0.0, 
                                            3.0, 
                                            8.0, 
                                            2900.0, 
                                            0.0, 
                                            47.6063, 
                                            -122.02, 
                                            2970.0, 
                                            5251.0, 
                                            12.0, 
                                            0.0, 
                                            0.0]]})

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}' > inferenceoutput.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput.df.json', orient="records")
df_result.loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   624  100   496  100   128  30468   7862 --:--:-- --:--:-- --:--:-- 39000


,time,out,metadata
0,1713800558148,{'variable': [659806.0]},"{'last_model': '{""model_name"":""xgb-house-price-estimator"",""model_sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}', 'pipeline_version': 'c6a1e945-7de0-4f2c-addb-4f4746114a86', 'elapsed': [359571, 613994], 'dropped': [], 'partition': 'houseprice-edge-inline-demonstration'}"


### How to Replace a Pipeline Publish from a New Pipeline

For this example, we will create a separate pipeline than the previous one with its own model.  This model will still accept the same inputs and output fields as the models previously deployed.  This is not a requirement - completely different models with different inputs and output schemas can be used; this example is used for simplicity.

For this set of examples, we will upload the third model, `./models/gbr_model.onnx` and create a new pipeline named `new-edge-inline-replacement`.  This model is added as a pipeline step and deployed.

In [16]:
housing_model_challenger02 = (wl.upload_model("gbr-house-price-estimator", 
                                        './models/gbr_model.onnx', 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

new_pipeline = wl.build_pipeline("new-edge-inline-replacement")

# undeploy if used before
new_pipeline.undeploy()
# clear the steps if used before
new_pipeline.clear()

new_pipeline.add_model_step(housing_model_challenger02)

#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

new_pipeline.deploy(deployment_config = deploy_config)
new_pipeline.steps()

Waiting for deployment - this will take up to 45s ............ ok


[{'ModelInference': {'models': [{'name': 'gbr-house-price-estimator', 'version': '90877932-73b5-4fd8-aa16-ed9c56e68a93', 'sha': 'ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a'}]}}]

In [17]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 
                                                      2.5, 
                                                      2900.0, 
                                                      5505.0, 
                                                      2.0, 
                                                      0.0, 
                                                      0.0, 
                                                      3.0, 
                                                      8.0, 
                                                      2900.0, 
                                                      0.0, 
                                                      47.6063, 
                                                      -122.02, 
                                                      2970.0, 
                                                      5251.0, 
                                                      12.0, 
                                                      0.0, 
                                                      0.0]]})
result = new_pipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2025-07-11 19:24:03.874,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[704901.9],0


#### Replace Pipeline Publish with Publish from Different Pipepline

We will now publish the new pipeline, and replace the previous edge location publish from our original pipeline.  The last publish is labeled `new_pub` is labeled `new_pipeline_pub`.

In [18]:
new_pipeline_pub = new_pipeline.publish(replaces=[new_pub])
new_pipeline_pub

Waiting for pipeline publish... It may take up to 600 sec.
... Published.blishing.


ID,87
Pipeline Name,new-edge-inline-replacement
Pipeline Version,5faddaaa-a642-4992-837b-83189b996a7c
Status,Published
Workspace Id,1450
Workspace Name,in-line-edge-replacement-demo
Edges,houseprice-edge-inline-demo
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/new-edge-inline-replacement:5faddaaa-a642-4992-837b-83189b996a7c
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/new-edge-inline-replacement
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:100156db8f3cb97b869fa4aead917ddbd244975d964f59c0d57af7f9ce9ca045


Once complete, we update the edge deployment with the new replacement `docker run` or `helm` based deployment commands from the configuration.

With the edge location replacement complete with the new pipeline, we list the edge locations from the original pipeline and the new one to show the edge location is transferred to the new pipeline.

In [19]:
display(mainpipeline.list_edges())

ID,Name,Publish ID,Created At,Tags,CPUs,Memory,SPIFFE ID
ca6018e1-2239-4511-958f-2aadb88cc92f,houseprice-edge-inline-demonstration,19,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/ca6018e1-2239-4511-958f-2aadb88cc92f
51fe06d6-d194-410c-ade5-8d9a60bd3da6,my-edge-demo,20,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/51fe06d6-d194-410c-ade5-8d9a60bd3da6
20db9fdb-2dde-4a18-bb75-2d62343d0a28,my-edge-demo-1,21,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/20db9fdb-2dde-4a18-bb75-2d62343d0a28
57a1dc90-4baf-4548-ae33-e3186b5672ea,my-edge-demo-2,22,None,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/57a1dc90-4baf-4548-ae33-e3186b5672ea


In [20]:
display(new_pipeline.list_edges())

ID,Name,Publish ID,Created At,Tags,CPUs,Memory,SPIFFE ID
619aefe2-5c23-48e1-a256-fc807d452d0f,houseprice-edge-inline-demo,87,2025-07-11T19:22:21.621626+00:00,[],4.0,3Gi,wallaroo.ai/ns/deployments/edge/619aefe2-5c23-48e1-a256-fc807d452d0f


#### Sample Inference Requests with New Pipeline Publish on Edge Location

We deploy the new publish on the edge device either with the **Docker Run Command** or the **Helm Install Command**, with whatever modifications are required on the edge device.  Once complete, we run through the `/models` and `/pipelines` endpoints, then perform an inference request.  We will see the name of the new model - and the new pipeline used for the same edge location.

In [26]:
!curl testboy.local:8080/models

{"models":[{"sha":"ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a","name":"gbr-house-price-estimator","version":"18b322c6-7681-44c5-a607-d8ec1f021651","status":"Running"}]}

In [27]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"new-edge-inline-replacement","version":"455a7840-08c3-43bb-b6a6-7535894e6055","status":"Running"}]}

In [28]:
# new inference

import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 
                                            2.5, 
                                            2900.0, 
                                            5505.0, 
                                            2.0, 
                                            0.0, 
                                            0.0, 
                                            3.0, 
                                            8.0, 
                                            2900.0, 
                                            0.0, 
                                            47.6063, 
                                            -122.02, 
                                            2970.0, 
                                            5251.0, 
                                            12.0, 
                                            0.0, 
                                            0.0]
                                            ]
                                })

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}' > inferenceoutput.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput.df.json', orient="records")
df_result.loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   624  100   496  100   128  25180   6498 --:--:-- --:--:-- --:--:-- 32842


,time,out,metadata
0,1713800828863,{'variable': [704901.9]},"{'last_model': '{""model_name"":""gbr-house-price-estimator"",""model_sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a""}', 'pipeline_version': '455a7840-08c3-43bb-b6a6-7535894e6055', 'elapsed': [337802, 597249], 'dropped': [], 'partition': 'houseprice-edge-inline-demonstration'}"


With the tutorial complete, we undeploy the pipelines and return the resources back to the cluster.

In [21]:
mainpipeline.undeploy()
new_pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ................................... ok
.................................... okke up to 45s 


name,new-edge-inline-replacement
created,2025-07-11 19:23:49.610108+00:00
last_updated,2025-07-11 19:24:04.018292+00:00
deployed,False
workspace_id,1450
workspace_name,in-line-edge-replacement-demo
arch,x86
accel,none
tags,
versions,"5faddaaa-a642-4992-837b-83189b996a7c, 0e77ba68-ff24-44a5-9c2a-6147f61766f2, af4501c6-5632-4125-9fbb-6428e847388d"
steps,gbr-house-price-estimator
